In [1]:
from itertools import izip_longest

def composed_product(P, Q):
    '''
    Composed product of P and Q
    '''
    A = P.parent()
    x = A.gen()
    m, n = P.degree(), Q.degree()
    dual = lambda R, d: R.derivative().reverse().multiplication_trunc(R.reverse().inverse_series_trunc(d), d)
    lift = A([(a*b) for a,b in izip_longest(dual(P, 2*m*n+1), dual(Q, 2*m*n+1), fillvalue=0)])
    R = lift.rational_reconstruct(x^(2*m*n+1), m*n, m*n)[1].reverse().monic()
    assert R.degree() == m*n
    return R

In [2]:
# Test
A.<x> = GF(5)[]

B.<X,Y> = GF(5)[]

for i in range(20):
    n = randrange(2,10)
    m = randrange(1,4)
    P = A.irreducible_element(n)
    Q = A.irreducible_element(n+m)
    R = composed_product(P, Q)
    assert R.degree() == P.degree() * Q.degree()
    assert P(X).resultant(Q.reverse()(X*Y)).univariate_polynomial()(x).reverse() == R

In [3]:
def primary_irred(A, q, e):
    return A.irreducible_element(q^e)
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

In [4]:
def irred(k, n):
    A = PolynomialRing(k, 'x')
    P = A.gen() - 1
    for q, e in n.factor():
        P = composed_product(P, primary_irred(A, q, e))
    return P

In [5]:
def complete_algebra(k, n):
    
    q = k.order()
    P = irred(k, q^n-1)
    
    m = GF(q^n).multiplicative_generator().minimal_polynomial()
    K.<z> = GF(q^(q^n-1), modulus = P, check_irreducible = False)
    A = K['z']
    h = A(m)
    
    return sage.rings.polynomial.polynomial_quotient_ring.PolynomialQuotientRing_domain(A, h, 'z')

In [6]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

def norm_r(x, a, b):
    d = b//a
    return product(frob_r(x, a*j) for j in range(d))

def nth_root_r(x, n):
    A = x.parent()
    k = A.base_ring().base_ring()
    C = k.extension(A.modulus().change_ring(k), 'x')
    y = C([k(a) for a in x.list()])
    return A(y.nth_root(n))

In [7]:
"""
Express `elem` in the base `newbase`. Outputs a column matrix.
"""
def base_change(elem, newbase, d2 = None):
    k = elem.base_ring()
    d1 = elem.parent().degree()
    
    if d2 == None:
        d2 = d1
        
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (newbase^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S = MatrixSpace(k, d1, 1)
    B = S(elem.list())
    X = A.solve_right(B)
    return X

In [8]:
def is_h90(a, zeta):
    return a != 0 and frob_l(a) == zeta * a

def solve_h90(A, z = None):
    x = A(1)
    n = A.base_ring().degree()
    if z == None:
        z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

def solve_h90bis(A, z = None, more = False, l = None):
    x = A(1)
    n = A.base_ring().degree()
    if l == None:
        l = n
    if z == None:
        z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        y = x
        zinv = z^-1
        zz = A.one()
        for i in range(1, n):
            zz *= zinv
            y = frob_l(y)
            x += y*zz
    
    if not more:
        return x
    else:
        k = A.base_ring().base_ring()
        t = (A.gen()^A.degree())/(x^l)
        return x*nth_root_r(t, l)

def test_h90(A, x, n = 10):
    z = A.gen()
    x1, x2 = x, x
    x1 = sum(frob_l(x1, i)*z^-i for i in range(n))
    
    y = x
    zinv = z^-1
    zz = A.one()
    for i in range(1, n):
        zz *= zinv
        y = frob_l(y)
        x2 += y*zz
    
    return x1 == x2

def time_h90(A, x, n = 10):
    z = A.gen()
    return sum(frob_l(x, i)*z^-i for i in range(n))

def time_h90bis(A, x, n = 10):
    z = A.gen()
    y = x
    zinv = z^-1
    zz = A.one()
    for i in range(1, n):
        zz *= zinv
        y = frob_l(y)
        x += y*zz
    return x

In [9]:
def trans(x, v):
    n, l = v.dimensions()
    for j in range(l):
        L = x[j].polynomial().list()
        for i in range(len(L)):
            v[i, j] = L[i]
    return v
    

def frob_lbis(x, M):
    
    A = x.parent()
    n = A.base_ring().degree()
    l = A.degree()
    V = MatrixSpace(A.base_ring().base_ring(), n, l)
    v = V()

    %time v = trans(x, v) # 90+ % spent here ... too bad
    Y = M*v
    
    return Y

In [10]:
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

def compute_emb(h1, h2):
    
    A, B = h1.parent(), h2.parent()
    a, b = A.degree(), B.degree()
    l, m = h1.base_ring().degree(), h2.base_ring().degree()

    p = h1.parent().characteristic()
    
    zeta_l, ZETA_L = A.gen()^((p^a-1)/l), B.gen()^((p^b-1)/l)

    u = base_change(h1, zeta_l)[0, 0]
    v = base_change(h2, ZETA_L, a)[0, 0]
    return compute_map(u, v)

In [11]:
def compute_inv_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = a.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, b.parent().degree(), 1)

    return lambda x : K(C.solve_right(S(fflist(x))).column(0))

def compute_inv_emb(h1, h2):
    
    A, B = h1.parent(), h2.parent()
    a, b = A.degree(), B.degree()
    l, m = h1.base_ring().degree(), h2.base_ring().degree()

    p = h1.parent().characteristic()
    
    zeta_l, ZETA_L = A.gen()^((p^a-1)/l), B.gen()^((p^b-1)/l)

    u = base_change(h1, zeta_l)[0, 0]
    v = base_change(h2, ZETA_L, a)[0, 0]
    return compute_inv_map(u, v)

In [12]:
def allombert(hl, hm):
    Al, Am = hl.parent(), hm.parent()
    a, b = Al.degree(), Am.degree()
    za = Am.gen()^((p^b-1)/(p^a-1))
    k = Al.base_ring().base_ring()
    R = k['x']
    l, m = Al.base_ring().degree(), Am.base_ring().degree()
    al, am = Am(R((hl^(l)).list()).subs(za)), hm^(m)
    r = nth_root_r(al/am, l)
    return r * hm^(m//l)

def factor_refinement(l, m):
    
    t = gcd(l, m)
    u, v = m//t, t
    d = gcd(u, v)
    
    while d != 1: 
        u, v = u//d, v*d
        d = gcd(u, v)
    
    return v, u
"""
Return N, e, t such that Norm( (alpha_N)^e ) = (a_l)^t · alpha_l
and m | N.
"""
def height(p, l, m):
    k = GF(p)
    a, b = cyclo_deg(k, l), cyclo_deg(k, m)
    S = p^a-1
    x, y = m//l, (p^b-1)//S
    
    xl, (yl, ym) = factor_refinement(l, x)[0], factor_refinement(l, y)
    t = (yl/xl).denominator()
    
    u = factor_refinement(l, m)[1]
        
    N = l*yl*t*u
    f = ym/u
    
    e = inverse_mod(f.numerator(), l)
    
    return N, t*e*f.denominator(), ((f.numerator()*e)//l)%S

def test_height(p, l, m):
    k = GF(p)
    R = k['z']
    N, e, c = height(p, l, m)
    a, b = cyclo_deg(k, l), cyclo_deg(k, m)
    Fpb = GF(p^b)
    zb = Fpb.multiplicative_generator()
    Pb = zb.minimal_polynomial()
    Pa = (zb^((p^b-1)//(p^a-1))).minimal_polynomial()
    Ab = PolynomialQuotientRing(GF(p^N)["x"], Pb.change_ring(GF(p^N)))
    Aa = PolynomialQuotientRing(GF(p^l)["x"], Pa.change_ring(GF(p^l)))
    
    hl = solve_h90bis(Aa, Aa.gen()^((p^a-1)//l), True)
    hN = solve_h90bis(Ab, Ab.gen()^((p^b-1)//N), True)
        
    CN = k.extension(Pb, 'z')
    za = CN.gen()^((p^b-1)//(p^a-1))
    al = Ab(R((hl^l).list()).subs(za))
    
    HL = allombert(hl, hN)
    
    t = (al^c)^-1 * (norm_r(hN, a, b))^e
    
    return Aa, Ab, t, HL, hl, hN

In [13]:
def test_triplet_same_r(p, l, m, n): 
    
    k = GF(p)
    
    assert cyclo_deg(k, l) == cyclo_deg(k, m) == cyclo_deg(k, n), "Not the same right part"
    
    # COMPUTATION OF CONSTANTS AND ZETAS
    
    R = k['z']
    a = cyclo_deg(k, l)
    Fpa = GF(p^a)
    za = Fpa.multiplicative_generator()
    Pa = za.minimal_polynomial()

    # COMPUTATION OF THE ALGEBRAS
    
    Al = PolynomialQuotientRing(GF(p^l)["x"], Pa.change_ring(GF(p^l)))
    Am = PolynomialQuotientRing(GF(p^m)["x"], Pa.change_ring(GF(p^m)))
    An = PolynomialQuotientRing(GF(p^n)["x"], Pa.change_ring(GF(p^n)))
    
    # COMPUTATION OF H90 SOLUTIONS
    
    hl = solve_h90bis(Al, Al.gen()^((p^a-1)//l), True)
    hm = solve_h90bis(Am, Am.gen()^((p^a-1)//m), True)
    hn = solve_h90bis(An, An.gen()^((p^a-1)//n), True)
    
    # COMPUTATION OF THE EMBEDDINGS
    
    f = compute_emb(hl, hm^(m/l))
    g = compute_emb(hm, hn^(n/m))
    h = compute_emb(hl, hn^(n/l))
    
    # TESTS
    
    x = GF(p^l).gen()
    r = GF(p^l).random_element()
    
    assert g(f(x)) == h(x), "Not compatible"
    assert g(f(r)) == h(r), "Not compatible"
    assert h(x).minpoly() == x.minpoly() == f(x).minpoly(), "Not a morphism"
    
    return f, g, h

def test_same_r(p, l, m):
    k = GF(p)
    R = k['z']
    a = cyclo_deg(k, l)
    Fpa = GF(p^a)
    za = Fpa.multiplicative_generator()
    Pa = za.minimal_polynomial()

    Am = PolynomialQuotientRing(GF(p^m)["x"], Pa.change_ring(GF(p^m)))
    Al = PolynomialQuotientRing(GF(p^l)["x"], Pa.change_ring(GF(p^l)))
    
    
    hl = solve_h90bis(Am, Am.gen()^((p^a-1)//l), True, l)
    hm = solve_h90bis(Am, Am.gen()^((p^a-1)//m), True)
    
    C = k.extension(Pa, 'z')
    al = hl^l # Am(R((hl^l).list()).subs(C.gen()))
    am = hm^m
    
    u = hl/(hm^(m//l))
    r = nth_root_r(u, m//l)
    
    t = r * hm
    
    return hl, t

In [14]:
def test_triplet(p, l, m, n):

    # FIRST COMPUTATIONS
    
    k = GF(p)
    R = k['z']
    a, b, c = cyclo_deg(k, l), cyclo_deg(k, m), cyclo_deg(k, n)
    
    # COMPUTATION OF THE ZETAS
    
    Fpc = GF(p^c)
    zc = Fpc.multiplicative_generator()
    Pc = zc.minimal_polynomial()
    Pb = (zc^((p^c-1)//(p^b-1))).minimal_polynomial()
    Pa = (zc^((p^c-1)//(p^a-1))).minimal_polynomial()
    
    # DIFFERENT HEIGHTS INVOLVED
    
    Nb, eb, cb = height(p, l, m)
    Nc, ec, cc = height(p, m, n)
    Nbc, ebc, cbc = height(p, l, n)
    
    assert Nc == Nbc, "Unexpected thing: not implemented"
    
    # COMPUTATION OF THE ALGEBRAS
    
    Ac = PolynomialQuotientRing(GF(p^Nc)["x"], Pc.change_ring(GF(p^Nc)))
    Ab = PolynomialQuotientRing(GF(p^Nb)["x"], Pb.change_ring(GF(p^Nb)))
    Aa = PolynomialQuotientRing(GF(p^l)["x"], Pa.change_ring(GF(p^l)))
    
    # SOME H90 SOLUTIONS
    
    hl = solve_h90bis(Aa, Aa.gen()^((p^a-1)//l), True)                 
    hNb = solve_h90bis(Ab, Ab.gen()^((p^b-1)//Nb), True)
    hNc = solve_h90bis(Ac, Ac.gen()^((p^c-1)//Nc), True)
    
    # SOME CONSTANTS TRANSLATIONS
    
    CNb = k.extension(Pb, 'z')
    za = CNb.gen()^((p^b-1)//(p^a-1))
    al = Ab(R((hl^l).list()).subs(za))
    
    CNc = k.extension(Pc, 'z')
    ZA = CNc.gen()^((p^c-1)//(p^a-1))
    ZB = CNc.gen()^((p^c-1)//(p^b-1))
    AL = Ac(R((hl^l).list()).subs(ZA))
    
    if (Nb > m):
        Am = PolynomialQuotientRing(GF(p^m)["x"], Pb.change_ring(GF(p^m)))
        hm = solve_h90bis(Am, Am.gen()^((p^b-1)//m), True)
        AM = Ac(R((hm^m).list()).subs(ZB))
    else:
        AM = Ac(R((hNb^Nb).list()).subs(ZB))
        
    # COMPUTATIONS OF THE EMBEDDINGS
    
    HLb = (al^cb)^-1 * norm_r(hNb, a, b)^eb
    HLc = (AL^cbc)^-1 * norm_r(hNc, a, c)^ebc
    HM = (AM^cc)^-1 * norm_r(hNc, b, c)^ec
    
    ff = compute_emb(hl, HLb)
    if (Nb > m):
        gg = compute_emb(hm, HM)
    else:
        gg = compute_emb(hNb, HM)
    hh = compute_emb(hl, HLc)
    
    if (Nb > m):
        f = lambda x : compute_inv_emb(hm, hNb^(Nb/m))(ff(x))
    else:
        f = ff
        
    if (Nc > n):
        An = PolynomialQuotientRing(GF(p^n)["x"], Pc.change_ring(GF(p^n)))
        hn = solve_h90bis(An, An.gen()^((p^c-1)//n), True)
        inv = compute_inv_emb(hn, hNc^(Nc/n))
        g = lambda x : inv(gg(x))
        h = lambda x : inv(hh(x))
    else:
        g, h = gg, hh
    
    # SOME TESTS
    
    K = GF(p^l)
    x, r = K.gen(), K.random_element()
                              
    assert g(f(x)) == h(x)
    assert g(f(r)) == h(r)
    assert x.minpoly() == h(x).minpoly()
    assert r.minpoly() == h(r).minpoly()
                              
    return f, g, h

In [198]:
def make_context(n):
    
    global p, ZETAS, H90_ELEMENTS, EMBEDDINGS
    
    ZETAS = dict()
    H90_ELEMENTS = dict()
    EMBEDDINGS = dict()
    p = n
    
def compute_zetas(n):
    k = GF(p^n)
    zeta_pn = k.multiplicative_generator()
    
    for d in n.divisors():
        ZETAS[d] = (zeta_pn^((p^n-1)//(p^d-1))).minpoly()

In [199]:
def embed(K, L):
    
    k = K.prime_subfield()
    l, m = K.degree(), L.degree()
    a, b = cyclo_deg(k, l), cyclo_deg(k, m)
    
    assert b in ZETAS.keys(), "The corresponding zeta is not computed: not implemented"
    
    if (l, m) in EMBEDDINGS.keys():
        return EMBEDDINGS[(l, m)]
    
    if l in H90_ELEMENTS.keys():
        hl = H90_ELEMENTS[l]
    else:
        Al = PolynomialQuotientRing(GF(p^l)["x"], ZETAS[a].change_ring(GF(p^l)))
        hl = solve_h90bis(Al, Al.gen()^((p^a-1)//l), True)
        H90_ELEMENTS[l] = hl
    
    if m in H90_ELEMENTS.keys():
        hm = H90_ELEMENTS[m]
    else:
        Am = PolynomialQuotientRing(GF(p^m)["x"], ZETAS[b].change_ring(GF(p^m)))
        hm = solve_h90bis(Am, Am.gen()^((p^b-1)//m), True)
        H90_ELEMENTS[m] = hm

    if a == b:
        f = compute_emb(hl, hm^(m//l))
        EMBEDDINGS[(l, m)] = f
        return f
    else:
        R = k['z']
        N, e, c = height(p, l, m)
        C = k.extension(ZETAS[b], 'z')
        ZA = C.gen()^((p^b-1)//(p^a-1))
        
        if N > m:
            if N in H90_ELEMENTS.keys():
                hN = H90_ELEMENTS[N]
            else:
                AN = PolynomialQuotientRing(GF(p^N)["x"], ZETAS[b].change_ring(GF(p^N)))
                hN = solve_h90bis(AN, AN.gen()^((p^b-1)//N), True)
                H90_ELEMENTS[N] = hN

            al = (hN.parent())(R((hl^l).list()).subs(ZA))
            HL = (al^c)^-1 * norm_r(hN, a, b)^e
            g = compute_emb(hl, HL)
            inv = compute_inv_emb(hm, hN^(N//m))
            f = lambda x : inv(g(x))
            EMBEDDINGS[(l, m)] = f
            return f
        else:
            al = (hm.parent())(R((hl^l).list()).subs(ZA))
            HL = (al^c)^-1 * norm_r(hm, a, b)^e
            f = compute_emb(hl, HL)
            EMBEDDINGS[(l, m)] = f
            return f

# Tests

- **On triche** parce qu'on calcule d'abord les racines p^n-1 ièmes de l'unité par le haut, pour se simplifier la vie niveau compatibilité...
- On ne traite que les cas où on doit travailler avec des algèbres, *i.e.* les racines ne sont pas dans le corps de base
- Pas non plus de cas Artin-Schreier

In [276]:
MAX = 36
p = 17
make_context(p)
compute_zetas(MAX)
factor(p^(MAX)-1)

2^6 * 3^4 * 5 * 7 * 13 * 19 * 29 * 37 * 109 * 181 * 307 * 1423 * 2089 * 5653 * 83233 * 382069 * 1270657

In [277]:
k3 = GF(p^3)
k6 = GF(p^6)
k12 = GF(p^(12))
k24 = GF(p^24)
k5 = GF(p^5)
k10 = GF(p^10)
k20 = GF(p^20)
k21 = GF(p^21)
k42 = GF(p^42)

In [278]:
f3_12 = embed(k3, k12)
f3_6 = embed(k3, k6)
f6_24 = embed(k6, k24)
f12_24 = embed(k12, k24)
f6_24 = embed(k6, k24)
f6_12 = embed(k6, k12)
f5_10 = embed(k5, k10)
f5_20 = embed(k5, k20)
f10_20 = embed(k10, k20)
f3_21 = embed(k3, k21)
f21_42 = embed(k21, k42)
f3_42 = embed(k3, k42)
f6_42 = embed(k6, k42)

In [279]:
assert f3_12(k3.gen()) == f6_12(f3_6(k3.gen()))
assert f6_24(f3_6(k3.gen())) == f12_24(f3_12(k3.gen()))
assert f10_20(f5_10(k5.gen())) == f5_20(k5.gen())
assert f21_42(f3_21(k3.gen())) == f3_42(k3.gen())
assert f6_42(f3_6(k3.gen())) == f3_42(k3.gen())